<a href="https://colab.research.google.com/github/AyushiKashyapp/kg_generation_from_text/blob/main/KGfromText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Knowledge Graph from Text

A project to implement a pipeline for extracting a Knowledge Base from texts or online articles.

Building a Knowledge Graph from text using following steps:

1. Extract entities, **Named Entity Recognition (NER)**, to represent the **nodes** of the graph.

2. Extract relations between entities, **Relation Classification (RC)**, to represent the **edges** of the graph.

These steps will be implemented in an end-to-end model called **REBEL (Relation Extraction By End-to-end Language generation)**.

# REBEL

REBEL is a text2text model trained by BabelSpace by fine-tuning BART for translating a raw input sentence containing entities and implicit relations into a set of triples that explicitly refer to those relations.

# Steps to implement the Knowledge Graph Extraction Pipeline.

1. Load the Relation Extraction REBEL model.
2. Extract a knowledge base from short text.
3. Extract a knowledge base from large text.
4. Filter and normalize entities.
5. Extract a knowledge base from an article at a specific URL.
6. Extract a knowledge base from multiple URLs.
7. Visualize knowldege bases.

**Necessary Libraries**

- **transformers** : Load the REBEL mode.
- **wikipedia**    : Validate extracted entities by checking if they have a corresponding Wikipedia page.
- **newspaper**    : Parse articles from URLs.
- **GoogleNews**   : Read Google News latest articles about a topic.
- **pyvis**        : Graphs visualisation.


In [1]:
!pip install transformers==4.37.2

In [2]:
!pip install transformers wikipedia newspaper3k GoogleNews

In [3]:
!pip install pyvis==0.3.1

In [4]:
!pip install lxml==4.9.3

# needed to load the REBEL model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch

# wrapper for wikipedia API
import wikipedia

# scraping of web articles
from newspaper import Article, ArticleException

# google news scraping
from GoogleNews import GoogleNews

# graph visualization
from pyvis.network import Network

# show HTML in notebook
import IPython

**Load Relation Extraction Model:**
Load the pre-trained REBEL model and tokenizer.

In [5]:
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large", trust_remote_code=True)

/home/anshi/test/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/anshi/test/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


**From Short Text to KB**

A function that is able to parse the strings generated by REBEL and transform them into relation triples.

In [6]:
# from https://huggingface.co/Babelscape/rebel-large
def extract_relations_from_model_output(text):
    relations = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")
    for token in text_replaced.split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        relations.append({
            'head': subject.strip(),
            'type': relation.strip(),
            'tail': object_.strip()
        })
    return relations

A **knowledge base class**, made of a list of relations and has several methods to deal with adding new relations to the knowledge base or printing them.

In [8]:
# class KB():
#     def __init__(self):
#         self.relations = []

#     def are_relations_equal(self, r1, r2):
#         return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

#     def exists_relation(self, r1):
#         return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

#     def add_relation(self, r):
#         if not self.exists_relation(r):
#             self.relations.append(r)

#     def print(self):
#         print("Relations:")
#         for r in self.relations:
#             print(f"  {r}")

A function that returns a Knowledge Base obejct with relations extracted from a short test.

1. Initialise an empty knowledge base KB object.
2. Tokenise the input text.
3. Use REBEL to generate relations from the text.
4. Parse REBEL output and store relation triples into the knowledge base object.
5. Return the knowledge base object.

In [9]:
# def from_small_text_to_kb(text, verbose=False):
#     kb = KB()

#     model_inputs = tokenizer(text, max_length=512, padding=True, truncation=True,
#                             return_tensors='pt')
#     if verbose:
#         print(f"Num tokens: {len(model_inputs['input_ids'][0])}")

#     gen_kwargs = {
#         "max_length": 216,
#         "length_penalty": 0,
#         "num_beams": 3,
#         "num_return_sequences": 3
#     }
#     generated_tokens = model.generate(
#         **model_inputs,
#         **gen_kwargs,
#     )
#     decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

#     for sentence_pred in decoded_preds:
#         relations = extract_relations_from_model_output(sentence_pred)
#         for r in relations:
#             kb.add_relation(r)

#     return kb

Calling the function on a small text.

In [10]:
# text = "Napoleon Bonaparte (born Napoleone di Buonaparte; 15 August 1769 – 5 " \
# "May 1821), and later known by his regnal name Napoleon I, was a French military " \
# "and political leader who rose to prominence during the French Revolution and led " \
# "several successful campaigns during the Revolutionary Wars. He was the de facto " \
# "leader of the French Republic as First Consul from 1799 to 1804. As Napoleon I, " \
# "he was Emperor of the French from 1804 until 1814 and again in 1815. Napoleon's " \
# "political and cultural legacy has endured, and he has been one of the most " \
# "celebrated and controversial leaders in world history."

# kb = from_small_text_to_kb(text, verbose=True)
# kb.print()

Num tokens: 133
Relations:
  {'head': 'Napoleon Bonaparte', 'type': 'date of birth', 'tail': '15 August 1769'}
  {'head': 'Napoleon Bonaparte', 'type': 'date of death', 'tail': '5 May 1821'}
  {'head': 'Napoleon Bonaparte', 'type': 'participant in', 'tail': 'French Revolution'}
  {'head': 'Napoleon Bonaparte', 'type': 'conflict', 'tail': 'Revolutionary Wars'}
  {'head': 'Revolutionary Wars', 'type': 'part of', 'tail': 'French Revolution'}
  {'head': 'French Revolution', 'type': 'participant', 'tail': 'Napoleon Bonaparte'}
  {'head': 'Revolutionary Wars', 'type': 'participant', 'tail': 'Napoleon Bonaparte'}


**From Long Text to KB**

Dividing a long input text of 1000 tokens into eight shorter overlapping spans lon 128 tokens nd extract relations from each span. While doing so, adding some metadata to the extracted relations containing their span boundaries.

Modifying the KB methods to save the span boundaries as well. The relation dictionary has the keys:

- head : The subject of the relation.
- type : The relation type.
- tail : The object of the relation.
- meta : Adictionary containing meta information about the relation. This dictionary has spans key, whose value is the list of span boundaries where the relation has been found.

In [8]:
class KB():
    def __init__(self):
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r1):
        r2 = [r for r in self.relations
              if self.are_relations_equal(r1, r)][0]
        spans_to_add = [span for span in r1["meta"]["spans"]
                        if span not in r2["meta"]["spans"]]
        r2["meta"]["spans"] += spans_to_add

    def add_relation(self, r):
        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def print(self):
      print("Relations:")
      for r in self.relations:
        print(f"  {r}")

A function to manage longer texts by splitting them into spans.

In [7]:
# extract relations for each span and put them together in a knowledge base
def from_text_to_kb(text, span_length=128, verbose=False):
    # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) /
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks)
    }

    # generate relations
    num_return_sequences = 3
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": num_return_sequences
    }
    generated_tokens = model.generate(
        **inputs,
        **gen_kwargs,
    )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)

    # create kb
    kb = KB()
    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // num_return_sequences
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            relation["meta"] = {
                "spans": [spans_boundaries[current_span_index]]
            }
            kb.add_relation(relation)
        i += 1

    return kb

Calling function on a longer text of 726 tokens.

In [9]:
text = """
Napoleon Bonaparte (born Napoleone di Buonaparte; 15 August 1769 – 5 May 1821), and later known by his regnal name Napoleon I, was a French military and political leader who rose to prominence during the French Revolution and led several successful campaigns during the Revolutionary Wars. He was the de facto leader of the French Republic as First Consul from 1799 to 1804. As Napoleon I, he was Emperor of the French from 1804 until 1814 and again in 1815. Napoleon's political and cultural legacy has endured, and he has been one of the most celebrated and controversial leaders in world history. Napoleon was born on the island of Corsica not long after its annexation by the Kingdom of France.[5] He supported the French Revolution in 1789 while serving in the French army, and tried to spread its ideals to his native Corsica. He rose rapidly in the Army after he saved the governing French Directory by firing on royalist insurgents. In 1796, he began a military campaign against the Austrians and their Italian allies, scoring decisive victories and becoming a national hero. Two years later, he led a military expedition to Egypt that served as a springboard to political power. He engineered a coup in November 1799 and became First Consul of the Republic. Differences with the British meant that the French faced the War of the Third Coalition by 1805. Napoleon shattered this coalition with victories in the Ulm Campaign, and at the Battle of Austerlitz, which led to the dissolving of the Holy Roman Empire. In 1806, the Fourth Coalition took up arms against him because Prussia became worried about growing French influence on the continent. Napoleon knocked out Prussia at the battles of Jena and Auerstedt, marched the Grande Armée into Eastern Europe, annihilating the Russians in June 1807 at Friedland, and forcing the defeated nations of the Fourth Coalition to accept the Treaties of Tilsit. Two years later, the Austrians challenged the French again during the War of the Fifth Coalition, but Napoleon solidified his grip over Europe after triumphing at the Battle of Wagram. Hoping to extend the Continental System, his embargo against Britain, Napoleon invaded the Iberian Peninsula and declared his brother Joseph King of Spain in 1808. The Spanish and the Portuguese revolted in the Peninsular War, culminating in defeat for Napoleon's marshals. Napoleon launched an invasion of Russia in the summer of 1812. The resulting campaign witnessed the catastrophic retreat of Napoleon's Grande Armée. In 1813, Prussia and Austria joined Russian forces in a Sixth Coalition against France. A chaotic military campaign resulted in a large coalition army defeating Napoleon at the Battle of Leipzig in October 1813. The coalition invaded France and captured Paris, forcing Napoleon to abdicate in April 1814. He was exiled to the island of Elba, between Corsica and Italy. In France, the Bourbons were restored to power. However, Napoleon escaped Elba in February 1815 and took control of France.[6][7] The Allies responded by forming a Seventh Coalition, which defeated Napoleon at the Battle of Waterloo in June 1815. The British exiled him to the remote island of Saint Helena in the Atlantic, where he died in 1821 at the age of 51. Napoleon had an extensive impact on the modern world, bringing liberal reforms to the many countries he conquered, especially the Low Countries, Switzerland, and parts of modern Italy and Germany. He implemented liberal policies in France and Western Europe.
"""

kb = from_text_to_kb(text, verbose=True)
kb.print()

Input has 726 tokens
Input has 6 spans
Span boundaries are [[0, 128], [119, 247], [238, 366], [357, 485], [476, 604], [595, 723]]
Relations:
  {'head': 'Napoleon Bonaparte', 'type': 'date of birth', 'tail': '15 August 1769', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Napoleon Bonaparte', 'type': 'date of death', 'tail': '5 May 1821', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Napoleon Bonaparte', 'type': 'participant in', 'tail': 'French Revolution', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Napoleon Bonaparte', 'type': 'conflict', 'tail': 'Revolutionary Wars', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Revolutionary Wars', 'type': 'part of', 'tail': 'French Revolution', 'meta': {'spans': [[0, 128]]}}
  {'head': 'French Revolution', 'type': 'participant', 'tail': 'Napoleon Bonaparte', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Revolutionary Wars', 'type': 'participant', 'tail': 'Napoleon Bonaparte', 'meta': {'spans': [[0, 128]]}}
  {'head': 'French Revolution', 'type': 'country', 'tai

**Filter and Normalize Entities with Wikipedia**

**Entity Linking** using wikipedia library to check if the same entities have the same wikipedia page. If so, they are normalised to the title of Wikipedia page.

Modifying the KB class to store an entities dictionary with the entities of the stored relations. The keys are entity identifiers, and the value is a dictionary containing the Wikipedia page URL and it's summary.

Then adding a new relation, to check its entities with the wikipedia library.

In [11]:
# filter and normalize entities before adding them to the KB
class KB():
    def __init__(self):
        self.entities = {}
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r1):
        r2 = [r for r in self.relations
              if self.are_relations_equal(r1, r)][0]
        spans_to_add = [span for span in r1["meta"]["spans"]
                        if span not in r2["meta"]["spans"]]
        r2["meta"]["spans"] += spans_to_add

    def get_wikipedia_data(self, candidate_entity):
        try:
            page = wikipedia.page(candidate_entity, auto_suggest=False)
            entity_data = {
                "title": page.title,
                "url": page.url,
                "summary": page.summary
            }
            return entity_data
        except:
            return None

    def add_entity(self, e):
        self.entities[e["title"]] = {k:v for k,v in e.items() if k != "title"}

    def add_relation(self, r):
        # check on wikipedia
        candidate_entities = [r["head"], r["tail"]]
        entities = [self.get_wikipedia_data(ent) for ent in candidate_entities]

        # if one entity does not exist, stop
        if any(ent is None for ent in entities):
            return

        # manage new entities
        for e in entities:
            self.add_entity(e)

        # rename relation entities with their wikipedia titles
        r["head"] = entities[0]["title"]
        r["tail"] = entities[1]["title"]

        # manage new relation
        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def print(self):
        print("Entities:")
        for e in self.entities.items():
            print(f"  {e}")
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")

In [12]:
text = """
Napoleon Bonaparte (born Napoleone di Buonaparte; 15 August 1769 – 5 May 1821), and later known by his regnal name Napoleon I, was a French military and political leader who rose to prominence during the French Revolution and led several successful campaigns during the Revolutionary Wars. He was the de facto leader of the French Republic as First Consul from 1799 to 1804. As Napoleon I, he was Emperor of the French from 1804 until 1814 and again in 1815. Napoleon's political and cultural legacy has endured, and he has been one of the most celebrated and controversial leaders in world history. Napoleon was born on the island of Corsica not long after its annexation by the Kingdom of France.[5] He supported the French Revolution in 1789 while serving in the French army, and tried to spread its ideals to his native Corsica. He rose rapidly in the Army after he saved the governing French Directory by firing on royalist insurgents. In 1796, he began a military campaign against the Austrians and their Italian allies, scoring decisive victories and becoming a national hero. Two years later, he led a military expedition to Egypt that served as a springboard to political power. He engineered a coup in November 1799 and became First Consul of the Republic. Differences with the British meant that the French faced the War of the Third Coalition by 1805. Napoleon shattered this coalition with victories in the Ulm Campaign, and at the Battle of Austerlitz, which led to the dissolving of the Holy Roman Empire. In 1806, the Fourth Coalition took up arms against him because Prussia became worried about growing French influence on the continent. Napoleon knocked out Prussia at the battles of Jena and Auerstedt, marched the Grande Armée into Eastern Europe, annihilating the Russians in June 1807 at Friedland, and forcing the defeated nations of the Fourth Coalition to accept the Treaties of Tilsit. Two years later, the Austrians challenged the French again during the War of the Fifth Coalition, but Napoleon solidified his grip over Europe after triumphing at the Battle of Wagram. Hoping to extend the Continental System, his embargo against Britain, Napoleon invaded the Iberian Peninsula and declared his brother Joseph King of Spain in 1808. The Spanish and the Portuguese revolted in the Peninsular War, culminating in defeat for Napoleon's marshals. Napoleon launched an invasion of Russia in the summer of 1812. The resulting campaign witnessed the catastrophic retreat of Napoleon's Grande Armée. In 1813, Prussia and Austria joined Russian forces in a Sixth Coalition against France. A chaotic military campaign resulted in a large coalition army defeating Napoleon at the Battle of Leipzig in October 1813. The coalition invaded France and captured Paris, forcing Napoleon to abdicate in April 1814. He was exiled to the island of Elba, between Corsica and Italy. In France, the Bourbons were restored to power. However, Napoleon escaped Elba in February 1815 and took control of France.[6][7] The Allies responded by forming a Seventh Coalition, which defeated Napoleon at the Battle of Waterloo in June 1815. The British exiled him to the remote island of Saint Helena in the Atlantic, where he died in 1821 at the age of 51. Napoleon had an extensive impact on the modern world, bringing liberal reforms to the many countries he conquered, especially the Low Countries, Switzerland, and parts of modern Italy and Germany. He implemented liberal policies in France and Western Europe.
"""

kb = from_text_to_kb(text)
kb.print()

/home/anshi/test/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/anshi/test/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Entities:
  ('Napoleon', {'url': 'https://en.wikipedia.org/wiki/Napoleon', 'summary': "Napoleon Bonaparte (born Napoleone Buonaparte; 15 August 1769 – 5 May 1821), later known by his regnal name Napoleon I, was a French general and statesman who rose to prominence during the French Revolution and led a series of military campaigns across Europe during the French Revolutionary and Napoleonic Wars from 1796 to 1815. He led the French Republic as First Consul from 1799 to 1804, then ruled the French Empire as Emperor of the French from 1804 to 1814, and briefly again in 1815. He was King of Italy from 1805 to 1814 and Protector of the Confederation of the Rhine from 1806 to 1813.\nBorn on the island of Corsica to a family of Italian origin, Napoleon moved to mainland France in 1779 and was commissioned as an officer in the French Royal Army in 1785. He supported the French Revolution in 1789 and promoted its cause in Corsica. He rose rapidly through the ranks after winning the siege of To

**Extract Knowledge Base from Web Article**

Modifying the KB class to allow the knowledge base to manage the addition of relations and entities from articles around the web, and to keep track of where each relation comes from.

Along with relations, entities and sources, storing the URL of each article as key and a dictionary with keys article_title.

When adding a new relation to the knowledge base, the relation meta field is now a dictionary with article URLs as keys, and another dictionary containing the spans as value.

In [10]:

# class KB():
#     def __init__(self):
#         self.entities = {} # { entity_title: {...} }
#         self.relations = [] # [ head: entity_title, type: ..., tail: entity_title,
#           # meta: { article_url: { spans: [...] } } ]
#         self.sources = {} # { article_url: {...} }

#     def are_relations_equal(self, r1, r2):
#         return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

#     def exists_relation(self, r1):
#         return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

#     def merge_relations(self, r2):
#         r1 = [r for r in self.relations
#               if self.are_relations_equal(r2, r)][0]

#         # if different article
#         article_url = list(r2["meta"].keys())[0]
#         if article_url not in r1["meta"]:
#             r1["meta"][article_url] = r2["meta"][article_url]

#         # if existing article
#         else:
#             spans_to_add = [span for span in r2["meta"][article_url]["spans"]
#                             if span not in r1["meta"][article_url]["spans"]]
#             r1["meta"][article_url]["spans"] += spans_to_add

#     def get_wikipedia_data(self, candidate_entity):
#         try:
#             page = wikipedia.page(candidate_entity, auto_suggest=False)
#             entity_data = {
#                 "title": page.title,
#                 "url": page.url,
#                 "summary": page.summary
#             }
#             return entity_data
#         except:
#             return None

#     def add_entity(self, e):
#         self.entities[e["title"]] = {k:v for k,v in e.items() if k != "title"}

#     def add_relation(self, r, article_title, article_publish_date):
#         # check on wikipedia
#         candidate_entities = [r["head"], r["tail"]]
#         entities = [self.get_wikipedia_data(ent) for ent in candidate_entities]

#         # if one entity does not exist, stop
#         if any(ent is None for ent in entities):
#             return

#         # manage new entities
#         for e in entities:
#             self.add_entity(e)

#         # rename relation entities with their wikipedia titles
#         r["head"] = entities[0]["title"]
#         r["tail"] = entities[1]["title"]

#         # add source if not in kb
#         article_url = list(r["meta"].keys())[0]
#         if article_url not in self.sources:
#             self.sources[article_url] = {
#                 "article_title": article_title,
#                 "article_publish_date": article_publish_date
#             }

#         # manage new relation
#         if not self.exists_relation(r):
#             self.relations.append(r)
#         else:
#             self.merge_relations(r)

#     def print(self):
#         print("Entities:")
#         for e in self.entities.items():
#             print(f"  {e}")
#         print("Relations:")
#         for r in self.relations:
#             print(f"  {r}")
#         print("Sources:")
#         for s in self.sources.items():
#             print(f"  {s}")

Modifying from_text_tokb function so that it prepares the relation meta field taking into account article URLs as well.

In [17]:
# extract text from url, extract relations and populate the KB
def from_text_to_kb(text, article_url, span_length=128, article_title=None,
                    article_publish_date=None, verbose=False):
  # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) /
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks)
    }

    # generate relations
    num_return_sequences = 3
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": num_return_sequences
    }
    generated_tokens = model.generate(
        **inputs,
        **gen_kwargs,
    )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)
    # create kb
    kb = KB()
    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // num_return_sequences
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            relation["meta"] = {
                article_url: {
                    "spans": [spans_boundaries[current_span_index]]
                }
            }
            kb.add_relation(relation, article_title, article_publish_date)
        i += 1

    return kb

Using newspaper library to download and parse articles from URLs and define a from_url_to_kb function. The library will automatically extract the article text, title, and publish date.

In [18]:
def get_article(url):
  article = Article(url)
  article.download()
  article.parse()
  return article

def from_url_to_kb(url):
  article = get_article(url)
  config = {
      "article_title" : article.title,
      "article_publish_date" : article.publish_date
  }
  kb = from_text_to_kb(article.text, article.url, **config)
  return kb

Testing function on article [Microstrategy chief: 'Bitcoin is going to go into the millions'](https://finance.yahoo.com/news/microstrategy-bitcoin-millions-142143795.html?guccounter=1&guce_referrer=aHR0cHM6Ly93d3cubmxwbGFuZXQub3JnLw&guce_referrer_sig=AQAAAEXrSBGkJoS8YSjo27jqq8oPk2P5Ti56j8-d42FV50TwnZLuniWVSWUBHEYMvIlGnHEL96wjr2nE3XNYEVqNZT1jy6vMNVaiacKgyfed4WKnA0AukooOZ8gFLL3tYAFbCK4hvhxvb50jy87WBO9tR2jsIJ2TGMBlkQYYT1TvXNbE)

In [19]:
url = "https://finance.yahoo.com/news/microstrategy-bitcoin-millions-142143795.html"
kb = from_url_to_kb(url)
kb.print()

/home/anshi/test/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/anshi/test/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Entities:
  ('MicroStrategy', {'url': 'https://en.wikipedia.org/wiki/MicroStrategy', 'summary': "MicroStrategy Incorporated, doing business as Strategy, is an American development company that provides business intelligence (BI), mobile software, and cloud-based services. Founded in 1989 by Michael J. Saylor, Sanju Bansal, and Thomas Spahr, the firm develops software to analyze internal and external data in order to make business decisions and to develop mobile apps. It is a public company headquartered in Tysons Corner, Virginia, in the Washington metropolitan area. Its primary business analytics competitors include SAP AG Business Objects, IBM Cognos, and Oracle Corporation's BI Platform. Saylor is the Executive Chairman and, from 1989 to 2022, was the CEO. \nSince 2020, the company's securities are widely considered to be a bitcoin proxy due to MicroStrategy's holdings of the cryptocurrency. The company's executive chairman has compared it to a bitcoin spot leveraged ETF, though it'

**Google News: Extract KB from Multiple Articles**

Creating the knowledge base from multiple articles, by extracting a separate knowledge base from each article and then merging all the knowledge bases together.

In [20]:
class KB():
    def __init__(self):
        self.entities = {} # { entity_title: {...} }
        self.relations = [] # [ head: entity_title, type: ..., tail: entity_title,
          # meta: { article_url: { spans: [...] } } ]
        self.sources = {} # { article_url: {...} }

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r2):
        r1 = [r for r in self.relations
              if self.are_relations_equal(r2, r)][0]

        # if different article
        article_url = list(r2["meta"].keys())[0]
        if article_url not in r1["meta"]:
            r1["meta"][article_url] = r2["meta"][article_url]

        # if existing article
        else:
            spans_to_add = [span for span in r2["meta"][article_url]["spans"]
                            if span not in r1["meta"][article_url]["spans"]]
            r1["meta"][article_url]["spans"] += spans_to_add

    def get_wikipedia_data(self, candidate_entity):
        try:
            page = wikipedia.page(candidate_entity, auto_suggest=False)
            entity_data = {
                "title": page.title,
                "url": page.url,
                "summary": page.summary
            }
            return entity_data
        except:
            return None

    def add_entity(self, e):
        self.entities[e["title"]] = {k:v for k,v in e.items() if k != "title"}

    def add_relation(self, r, article_title, article_publish_date):
        # check on wikipedia
        candidate_entities = [r["head"], r["tail"]]
        entities = [self.get_wikipedia_data(ent) for ent in candidate_entities]

        # if one entity does not exist, stop
        if any(ent is None for ent in entities):
            return

        # manage new entities
        for e in entities:
            self.add_entity(e)

        # rename relation entities with their wikipedia titles
        r["head"] = entities[0]["title"]
        r["tail"] = entities[1]["title"]

        # add source if not in kb
        article_url = list(r["meta"].keys())[0]
        if article_url not in self.sources:
            self.sources[article_url] = {
                "article_title": article_title,
                "article_publish_date": article_publish_date
            }

        # manage new relation
        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def merge_with_kb(self, kb2):
        for r in kb2.relations:
            article_url = list(r["meta"].keys())[0]
            source_data = kb2.sources[article_url]
            self.add_relation(r, source_data["article_title"],
                              source_data["article_publish_date"])

    def print(self):
        print("Entities:")
        for e in self.entities.items():
            print(f"  {e}")
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")
        print("Sources:")
        for s in self.sources.items():
            print(f"  {s}")



Using GoogleNews library to get the URLs of recent news articles about a specific topic. The multiple URLs are fed to the from_url_to_kb function, to extract a knowledge base from each article and then merge them together.

In [21]:
# get news links from google news
def get_news_links(query, lang="en", region="US", pages=1, max_links=100000):
    googlenews = GoogleNews(lang=lang, region=region)
    googlenews.search(query)
    all_urls = []
    for page in range(pages):
        googlenews.get_page(page)
        all_urls += googlenews.get_links()
    return list(set(all_urls))[:max_links]

# build a KB from multiple news links
def from_urls_to_kb(urls, verbose=False):
    kb = KB()
    if verbose:
        print(f"{len(urls)} links to visit")
    for url in urls:
        if verbose:
            print(f"Visiting {url}...")
        try:
            kb_url = from_url_to_kb(url)
            kb.merge_with_kb(kb_url)
        except ArticleException:
            if verbose:
                print(f"  Couldn't download article at url {url}")
    return kb

In [22]:
news_links = get_news_links("Google", pages=1, max_links=100)
kb = from_urls_to_kb(news_links, verbose=True)
kb.print()

20 links to visit
Visiting https://blog.google/outreach-initiatives/entrepreneurs/growth-academy-ai-health-2025/&ved=2ahUKEwjkrLKg4MWMAxXsRmwGHWzwM-4QxfQBegQIAhAC&usg=AOvVaw1XmLbeFriSuSKtCGKQNwAy...
  Couldn't download article at url https://blog.google/outreach-initiatives/entrepreneurs/growth-academy-ai-health-2025/&ved=2ahUKEwjkrLKg4MWMAxXsRmwGHWzwM-4QxfQBegQIAhAC&usg=AOvVaw1XmLbeFriSuSKtCGKQNwAy
Visiting https://www.idolgu.in/googles-biggest-deal-ever-32-billion-acquisition-of-cybersecurity-giant-wiz/&ved=2ahUKEwjkrLKg4MWMAxXsRmwGHWzwM-4QxfQBegQICBAC&usg=AOvVaw0175mX8gBRoL4we6ahBEka...
Visiting https://www.androidpolice.com/5-wear-os-annoyances-google-can-no-longer-ignore/&ved=2ahUKEwjkrLKg4MWMAxXsRmwGHWzwM-4QxfQBegQIARAC&usg=AOvVaw3x5lHukfOT-0AGUyRHkHQp...
  Couldn't download article at url https://www.androidpolice.com/5-wear-os-annoyances-google-can-no-longer-ignore/&ved=2ahUKEwjkrLKg4MWMAxXsRmwGHWzwM-4QxfQBegQIARAC&usg=AOvVaw3x5lHukfOT-0AGUyRHkHQp
Visiting https://m.economictim

**Visualise Knowledge Base**

Using pyvis to visualise the knowledge Base:

- Initialise an empty directed pyvis network.
- Add the knowledge base entities as nodes.
- Add the knowledge base relations as edges.
- Save the network in an HTML file.

In [13]:
# from KB to HTML visualization
def save_network_html(kb, filename="network.html"):
    # create network
    net = Network(directed=True, width="auto", height="700px", bgcolor="#eeeeee")

    # nodes
    color_entity = "#00FF00"
    for e in kb.entities:
        net.add_node(e, shape="circle", color=color_entity)

    # edges
    for r in kb.relations:
        net.add_edge(r["head"], r["tail"],
                    title=r["type"], label=r["type"])

    # save network
    net.repulsion(
        node_distance=200,
        central_gravity=0.2,
        spring_length=200,
        spring_strength=0.05,
        damping=0.09
    )
    net.set_edge_smooth('dynamic')
    net.show(filename)

In [14]:
filename = "network_3_google.html"
save_network_html(kb, filename=filename)